In [ ]:
%pip install supervision
%pip -q install roboflow
%pip -q install ultralytics==8.0.28


In [2]:
from google.colab import drive
drive.mount('/content/drive')
dataset_location = "/content/drive/MyDrive/CsiLab-BrainTumor-Detection-2"

Mounted at /content/drive


In [3]:
import supervision as sv



In [4]:
from ultralytics import YOLO
from PIL import Image
def predict_img(img_path, model_path):

  model = YOLO(model_path)
  source = img_path

  results = model(source)
  for r in results:
    try:
      return r.boxes.xywhn.cpu().numpy()[0]
    except Exception as e:
      return [0,0,0,0]

In [5]:
def calcula_iou_numpy(predict, label):
    # caixa1 e caixa2 devem ser arrays NumPy no formato [x, y, largura, altura]
    x1, y1, w1, h1 = predict
    id2, x2, y2, w2, h2 = label

    # Calcula as coordenadas dos cantos da interseção
    x_intersecao = max(x1, x2)
    y_intersecao = max(y1, y2)
    largura_intersecao = min(x1 + w1, x2 + w2) - x_intersecao
    altura_intersecao = min(y1 + h1, y2 + h2) - y_intersecao

    # Verifica se há interseção
    if largura_intersecao <= 0 or altura_intersecao <= 0:
        return 0.0

    # Calcula a área da interseção e da união
    area_intersecao = largura_intersecao * altura_intersecao
    area_uniao = w1 * h1 + w2 * h2 - area_intersecao

    # Calcula o IoU
    iou = area_intersecao / area_uniao

    return iou

In [ ]:
import os
import numpy as np
source = '/content/drive/MyDrive/CsiLab-BrainTumor-Detection-2/test'
yolo_s = []
yolo_m = []
yolo_n = []
yolo_l = []
yolo_x = []



for filename in os.listdir(f"{source}/labels"):
    labels = np.loadtxt(os.path.join(f"{source}/labels", filename))
    images = os.path.join(f"{source}/images", filename).split('.txt')[0] + '.jpg'
    print(images)


    yolo_s.append(calcula_iou_numpy(predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/trainyolov8s/weights/best.pt'), labels))
    yolo_m.append(calcula_iou_numpy(predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train2yolov8m/weights/best.pt'), labels))
    yolo_n.append(calcula_iou_numpy(predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train5yolov8n/weights/best.pt'), labels))
    yolo_l.append(calcula_iou_numpy(predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train3yolov8L/weights/best.pt'), labels))
    yolo_x.append(calcula_iou_numpy(predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train4yolov8X/weights/best.pt'), labels))


In [7]:
from statistics import mean

print('Media de iou com yolov8s: ',mean(yolo_s))
print('Media de iou com yolov8m: ',mean(yolo_m))
print('Media de iou com yolov8n: ',mean(yolo_n))
print('Media de iou com yolov8l: ',mean(yolo_l))
print('Media de iou com yolov8x: ',mean(yolo_x))

Media de iou com yolov8s:  0.6655151926301139
Media de iou com yolov8m:  0.7919575251391155
Media de iou com yolov8n:  0.7833387240130778
Media de iou com yolov8l:  0.7803399691548843
Media de iou com yolov8x:  0.7836144900062253
